##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification with TensorFlow Lite model customization with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_customization/demo/image_classification.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/tensorflow_examples/lite/model_customization/demo/image_classification.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

A biblioteca de personalização do modelo simplifica o processo de adaptação e conversão de um modelo de rede neural do TensorFlow em dados de entrada específicos ao implantar esse modelo para aplicativos ML no dispositivo.

Este notebook mostra um exemplo de ponta a ponta que utiliza essa biblioteca de personalização de modelo para ilustrar a adaptação e conversão de um modelo de classificação de imagem comumente usado para classificar flores em um dispositivo móvel.

## Pré-requisitos

Para executar este exemplo, primeiro precisamos instalar os pacotes necessários para o servidor, incluindo o pacote de personalização de modelos que no github [repo] (https://github.com/tensorflow/examples).

In [0]:
%tensorflow_version 2.x
!pip install -q git+https://github.com/tensorflow/examples

Import the required packages.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tensorflow_examples.lite.model_customization.core.data_util.image_dataloader import ImageClassifierDataLoader
from tensorflow_examples.lite.model_customization.core.task import image_classifier
from tensorflow_examples.lite.model_customization.core.task.model_spec import efficientnet_b0_spec
from tensorflow_examples.lite.model_customization.core.task.model_spec import ImageModelSpec

import matplotlib.pyplot as plt

## Exemplo simples de ponta a ponta

Vamos usar algumas imagens para jogar com este exemplo simples de ponta a ponta. Você pode substituí-lo por suas próprias pastas de imagens. Centenas de imagens são um bom começo para a personalização do modelo, enquanto mais dados podem alcançar uma melhor precisão.

In [0]:
image_path = tf.keras.utils.get_file(
      'flower_photos',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      untar=True)

O exemplo consiste apenas em 4 linhas de código, como mostrado abaixo, cada uma representando uma etapa do processo geral.

1. Carregue dados de entrada específicos para um aplicativo ML no dispositivo.

In [0]:
data = ImageClassifierDataLoader.from_folder(image_path)

2. Personalize o modelo TensorFlow.

In [0]:
model = image_classifier.create(data)

3. Avalie o modelo.

In [0]:
loss, accuracy = model.evaluate()

4. Exporte para o modelo TensorFlow Lite.

In [0]:
model.export('image_classifier.tflite', 'image_labels.txt')

Após essas 4 etapas simples, poderíamos usar ainda o arquivo de modelo e o rótulo do modelo TensorFlow Lite em aplicativos no dispositivo, como em [classificação da imagem] (https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification ) aplicativo de referência.

## Processo detalhado

Atualmente, incluímos apenas os modelos MobileNetV2 e EfficientNetB0 como modelos pré-treinados para classificação de imagens. Mas é muito flexível adicionar novos modelos pré-treinados a esta biblioteca com apenas algumas linhas de código.


A seguir, passo a passo neste exemplo de ponta a ponta para mostrar mais detalhes.

### Etapa 1: carregar dados de entrada específicos para um aplicativo ML no dispositivo

O conjunto de dados da flor contém 3670 imagens pertencentes a 5 classes. Baixe a versão de arquivo morto do conjunto de dados e descompacte-a.

O conjunto de dados tem a seguinte estrutura de diretórios:

<pre>
<b>flower_photos</b>
|__ <b>daisy</b>
    |______ 100080576_f52e8ee070_n.jpg
    |______ 14167534527_781ceb1b7a_n.jpg
    |______ ...
|__ <b>dandelion</b>
    |______ 10043234166_e6dd915111_n.jpg
    |______ 1426682852_e62169221f_m.jpg
    |______ ...
|__ <b>roses</b>
    |______ 102501987_3cdb8e5394_n.jpg
    |______ 14982802401_a3dfb22afb.jpg
    |______ ...
|__ <b>sunflowers</b>
    |______ 12471791574_bb1be83df4.jpg
    |______ 15122112402_cafa41934f.jpg
    |______ ...
|__ <b>tulips</b>
    |______ 13976522214_ccec508fe7.jpg
    |______ 14487943607_651e8062a1_m.jpg
    |______ ...
</pre>

In [0]:
image_path = tf.keras.utils.get_file(
      'flower_photos',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      untar=True)

Use a classe `ImageClassifierDataLoader` para carregar dados.

Quanto ao método `from_folder ()`, ele pode carregar dados da pasta. Ele pressupõe que os dados de imagem da mesma classe estejam no mesmo subdiretório e o nome da subpasta seja o nome da classe. Atualmente, imagens codificadas em JPEG e imagens codificadas em PNG são suportadas.

In [0]:
data = ImageClassifierDataLoader.from_folder(image_path)

Mostre 30 exemplos de imagens com etiquetas.

In [0]:
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(data.dataset.take(30)):
  plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

Etapa 2: personalizar o modelo TensorFlow

Crie um modelo de classificador de imagem personalizado com base nos dados carregados. O modelo padrão é MobileNetV2.

In [0]:
model = image_classifier.create(data)

Veja a estrutura detalhada do modelo.

In [0]:
model.summary()

### Etapa 3: avaliar o modelo personalizado

Avalie o resultado do modelo, obtenha a perda e a precisão do modelo.

Por padrão, os resultados são avaliados nos dados de teste divididos no método `create`. Outros dados de teste também podem ser avaliados se servidos como parâmetro.

In [0]:
loss, accuracy = model.evaluate()

Podemos plotar os resultados previstos em 100 imagens de teste. Rótulos previstos com cor vermelha são os resultados previstos incorretos, enquanto outros estão corretos.

In [0]:
# Uma função auxiliar que retorna 'vermelho' / 'preto', dependendo de suas duas entradas
# parâmetro corresponde ou não.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'green'
  else:
    return 'red'

# Em seguida, plote 100 imagens de teste e seus rótulos previstos.
# Se um resultado de previsão for diferente do rótulo fornecido em "teste"
# dataset, destacaremos em vermelho.
plt.figure(figsize=(20, 20))
for i, (image, label) in enumerate(model.test_data.dataset.take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

# O pré-processamento deve permanecer o mesmo. Atualmente, apenas normalize cada valor de pixel para [0, 1] 
# e redimensione a imagem para [224, 224, 3].
  image, label = model.preprocess_image(image, label)
# Adicione a dimensão do lote e converta para float32 para corresponder à entrada do modelo
# formato de dados.
  image = tf.expand_dims(image, 0).numpy()

  predict_prob = model.model.predict(image)
  predict_label = np.argmax(predict_prob, axis=1)[0]

  ax.xaxis.label.set_color(get_label_color(predict_label,\
                                           label.numpy()))
  plt.xlabel('Predicted: %s' % model.test_data.index_to_label[predict_label])
plt.show()

Se a precisão não atender aos requisitos do aplicativo, é possível consultar [Uso avançado] (# scrollTo = zNDBP2qA54aK) para explorar alternativas, como mudar para um modelo maior, ajustar os parâmetros de re-treinamento etc.

Etapa 4: Exportar para o modelo TensorFlow Lite

Converta o modelo existente no formato de modelo TensorFlow Lite e salve os rótulos da imagem no arquivo de etiqueta.

In [0]:
model.export('flower_classifier.tflite', 'flower_labels.txt')

O arquivo de modelo e o rótulo do modelo TensorFlow Lite podem ser usados no aplicativo de referência [classificação da imagem] (https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification).

Quanto ao aplicativo de referência para Android como exemplo, podemos adicionar `flower_classifier.tflite` e` flower_label.txt` em [assets] (https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/ pasta android / app / src / main / assets). Enquanto isso, altere o nome do arquivo do rótulo em [code] (https://github.com/tensorflow/examples/blob/master/lite/examples/image_classification/android/app/src/main/java/org/tensorflow/lite/examples/ rating / tflite / ClassifierFloatMobileNet.java # L65) e nome do arquivo TensorFlow Lite em [code] (https://github.com/tensorflow/examples/blob/master/lite/examples/image_classification/android/app/src/main/ java / org / tensorflow / lite / exemplos / classificação / tflite / ClassifierFloatMobileNet.java # L60). Assim, poderíamos executar o modelo TensorFlow Lite flutuante reciclado no aplicativo Android.

Aqui, também demonstramos como usar os arquivos acima para executar e avaliar o modelo TensorFlow Lite.

In [0]:
# Read TensorFlow Lite model from TensorFlow Lite file.
with tf.io.gfile.GFile('flower_classifier.tflite', 'rb') as f:
  model_content = f.read()

# Read label names from label file.
with tf.io.gfile.GFile('flower_labels.txt', 'r') as f:
  label_names = f.read().split('\n')

# Initialze TensorFlow Lite inpterpreter.
interpreter = tf.lite.Interpreter(model_content=model_content)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

# Run predictions on each test image data and calculate accuracy.
accurate_count = 0
for i, (image, label) in enumerate(model.test_data.dataset):
    # Pre-processing should remain the same. Currently, just normalize each pixel value and resize image according to the model's specification.
    image, label = model.preprocess_image(image, label)
    # Add batch dimension and convert to float32 to match with the model's input
    # data format.
    image = tf.expand_dims(image, 0).numpy()

    # Run inference.
    interpreter.set_tensor(input_index, image)
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the label with highest
    # probability.
    predict_label = np.argmax(output()[0])
    # Get label name with label index.
    predict_label_name = label_names[predict_label]

    accurate_count += (predict_label == label.numpy())

accuracy = accurate_count * 1.0 / model.test_data.size
print('TensorFlow Lite model accuracy = %.4f' % accuracy)

Observe que o pré-processamento para inferência deve ser o mesmo que treinamento. Atualmente, o pré-processamento contém a normalização de cada valor de pixel e o redimensionamento da imagem de acordo com as especificações do modelo. Para o MobileNetV2, a imagem de entrada deve ser normalizada para `[0, 1]` e redimensionada para `[224, 224, 3]`.

## Uso avançado

A função `create` é a parte crítica desta biblioteca. Ele usa o aprendizado de transferência com um modelo pré-treinado semelhante ao [tutorial] (https://www.tensorflow.org/tutorials/images/transfer_learning).

A função `create` contém os seguintes passos:

1. Divida os dados em treinamento, validação e dados de teste de acordo com os parâmetros `validation_ratio` e` test_ratio`. O valor padrão de `validation_ratio` e` test_ratio` é `0.1` e` 0.1`.
2. Faça o download de um [Vetor de recurso de imagem] (https://www.tensorflow.org/hub/common_signatures/images#image_feature_vector) como modelo base no TensorFlow Hub. O modelo pré-treinado padrão é o MobileNetV2.
3. Adicione um cabeçote classificador com uma camada de dropout com `dropout_rate` entre a camada de head e o modelo pré-treinado. O padrão `dropout_rate` é` 0.2`.
4. Pré-processe os dados de entrada brutos. Atualmente, as etapas de pré-processamento incluem a normalização do valor de cada pixel da imagem para modelar a escala de entrada e redimensioná-lo para modelar o tamanho da entrada. O MobileNetV2 possui a escala de entrada `[0, 1]` e o tamanho da imagem de entrada `[224, 224, 3]`.
5. Alimente os dados no modelo do classificador. Por padrão, o número de épocas de treinamento é `2`, o tamanho do lote é` 32` e apenas o chefe do classificador é treinado.


Nesta seção, descrevemos vários tópicos avançados, incluindo a mudança para um modelo de classificação de imagem diferente, a alteração dos hiperparâmetros de treinamento etc.

## Change the model



### Altere para o modelo suportado nesta biblioteca.

Esta biblioteca suporta os modelos MobileNetV2 e EfficientNetB0 até agora. O modelo padrão é MobileNetV2.

[EfficientNets] (https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet) são uma família de modelos de classificação de imagens que podem obter precisão de última geração. O EfficinetNetB0 é um dos modelos EfficientNet que é pequeno e adequado para aplicações no dispositivo. É maior que o MobileNetV2, enquanto pode alcançar um melhor desempenho.

Poderíamos mudar o modelo para EfficientNetB0 apenas configurando o parâmetro `model_spec` como `fficientnet_b0_spec` no método` create`.

In [0]:
model = image_classifier.create(data, model_spec=efficientnet_b0_spec)

Evaluate the newly retrained EfficientNetB0 model to see the accuracy and loss in testing data.

In [0]:
loss, accuracy = model.evaluate()

### Alterar para o modelo no TensorFlow Hub

Além disso, também poderíamos mudar para outros novos modelos que inserem uma imagem e produzem um vetor de recurso no formato TensorFlow Hub.

Como o modelo [Inception V3] (https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1) como exemplo, poderíamos definir `inception_v3_spec`, que é um objeto do` ImageModelSpec` e contém a especificação do Inception Modelo V3.

Precisamos especificar o nome do modelo `name`, o URL do modelo do TensorFlow Hub` uri`, a versão TensorFlow do modelo `tf_version`. Enquanto isso, o valor padrão de `input_image_shape` é` [224, 224] `. Precisamos alterá-lo para `[299, 299]` para o modelo Inception V3.

In [0]:
inception_v3_spec = ImageModelSpec(
    name='inception_v3',
    uri='https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1',
    tf_version=1)
inception_v3_spec.input_image_shape = [299, 299]

Then, by setting parameter model_spec to `inception_v3_spec` in `create` method, we could retrain the Inception V3 model. 

The remaining steps are exactly same and we could get a customized InceptionV3 TensorFlow Lite model in the end.

### Change your own custom model

If we'd like to use the custom model that's not in TensorFlow Hub, we should create and export [ModelSpec](https://www.tensorflow.org/hub/api_docs/python/hub/ModuleSpec) in TensorFlow Hub.

Then start to define `ImageModelSpec` object like the process above.

In [0]:
model = image_classifier.create(data, epochs=5)

Evaluate the newly retrained model with 5 training epochs.

In [0]:
loss, accuracy = model.evaluate()